* SAMPLE CREATION *

In [ ]:
import cv2
import numpy as np

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect 100 samples of your face from webcam input
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Save file in specified directory with unique name
        file_name_path = './faces/user2/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 100: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")
cap.release()

*Train Model*

In [ ]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

# Get the training data we previously made
data_path = './faces/user2/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

# Create arrays for training data and labels
Training_Data, Labels = [], []

# Open training images in our datapath
# Create a numpy array for training data
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)

# Create a numpy array for both training data and labels
Labels = np.asarray(Labels, dtype=np.int32)

# Initialize facial recognizer
# model = cv2.face.createLBPHFaceRecognizer()
# NOTE: For OpenCV 3.0 use cv2.face.createLBPHFaceRecognizer()
# pip install opencv-contrib-python
# model = cv2.createLBPHFaceRecognizer()

model  = cv2.face_LBPHFaceRecognizer.create()
# Let's train our model 
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model trained sucessefully")
model.save("traning_face2.yml")

*Facial Recognition*

In [1]:
send_sk = False
send_nelson = False

In [6]:
import cv2
import numpy as np
import os
import time


face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
fname = "traning_face.yml"
if not os.path.isfile(fname):
  print("Please train the data first")
  exit(0)

def face_detector(img, size=0.5):
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img, []
    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi


# Open Webcam
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    image, face = face_detector(frame)
    
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        # Pass face to prediction model
        # "results" comprises of a tuple containing the label and the confidence value
        models  = cv2.face_LBPHFaceRecognizer.create()
        models.read(fname)
        results = models.predict(face)
        #..........
        
        elon_models  = cv2.face_LBPHFaceRecognizer.create()
        elon_models.read('traning_face2.yml')
        elon_results = elon_models.predict(face)
        #..........
        # harry_model.predict(face)
        
        if results[1] < 500:
            confidence = int( 100 * (1 - (results[1])/400) )
            display_string = str(confidence) + '% Confident it is User'
            
            #cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if confidence > 85:
            cv2.putText(image, "Hey SK", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            print('hi')
            if (send_sk == False):
                print("IN")
                sendEmail()
                sendwapp()
                send_sk = True
            
            
            
        
        #..........
        if elon_results[1] < 500:
            elon_confidence = int( 100 * (1 - (elon_results[1])/400) )
            display_string = str(elon_confidence) + '% Confident it is Neslson Mandela'
            
            #cv2.putText(image, display_string, (100, 240), cv2.FONT_HERSHEY_COMPLEX, 1, (255,120,150), 2)
        
        if elon_confidence > 85:
            cv2.putText(image, "Hey Mr Mandela", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow('Face Recognition', image )
            if (send_nelson == False):
                print("IN")
                createVol()
                createEc2()
                send_nelson = True
                
                
         #.......... 
        else:
            
            #cv2.putText(image, "X", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
            cv2.imshow('Face Recognition', image )

    except:
        cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.putText(image, "looking for face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()     

<>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-6-9069a7dd712c>:18: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if faces is ():


hi
IN
EMAIL SUCCESSFULLY SEND
In 11 seconds web.whatsapp.com will open and after 20 seconds message will be delivered
Whats App Successfully Sent!
IN
			==============V0lume created Successfully======================
			==============EC2 Created Successfully======================


*SEND EMAIL*

In [2]:
import smtplib
def sendEmail():
    fromaddr = '98thplayer@gmail.com'  
    toaddrs  = 'mailshashi122@gmail.com'  
    msg = 'Logged in using Your Face'  
    username = '98thplayer' 
    password = '*********'
    server = smtplib.SMTP('smtp.gmail.com', 587)  
    server.ehlo()
    server.starttls()
    server.login(username, password)  
    server.sendmail(fromaddr, toaddrs, msg)  
    print("EMAIL SUCCESSFULLY SEND")
    server.quit()

*SEND WHATSAPP*

In [3]:
# importing the module
import pywhatkit
import datetime
def sendwapp():
    now = datetime.datetime.now()
    hr = now.hour
    mi = now.minute

    # using Exception Handling to avoid 
    # unprecedented errors
    #try:

      # sending message to reciever
      # using pywhatkit
    pywhatkit.sendwhatmsg("+918934939855", 
                        "Hello from Python", 
                        hr, mi+1)
    print("Whats App Successfully Sent!")

   

In [4]:
#  CREATE VOLUME AND AWS instance

In [5]:
import os
def createVol():
    os.system("aws ec2 create-volume --availability-zone ap-south-1a --volume-type gp2 --size 1")
    print("\t\t\t==============V0lume created Successfully======================")
def createEc2():
    os.system("aws ec2 run-instances --image-id ami-011c99152163a87ae --instance-type t2.micro --key-name test_key_ec2")
    print("\t\t\t==============EC2 Created Successfully======================")
